<h1><center>Hito 9</center></h1>

<h1><center>Compressing the audio data with zlib</center></h1>

&nbsp;

<center>
Álvaro Garcı́a Garcı́a<br>
Álvaro José Martı́nez Sánchez<br>
José Francisco Castillo Berenguel<br>
</center>
    
    
<hr style="border:1px solid gray"> </hr>


# Introducción

Se presentaron dos problemas fundamentales de la transmisión por internet: jitter y byte rate. En el hito 8 se resolvió el problema del jitter  por medio de un buffer circular. En este hito se atenuarán los efectos del byte rate mediante la compresión de los paquetes que se envíen.




# Fundamentos

Para este hito se requirirá de un proceso de compresión. No obstante, dicho proceso de compresión puede mejorarse explotando la correlación estadística que existe dentro de cada canal. Para ello, se procederá con una serie de transformaciones que nos permitan dicho aprovechamiento. 


## Zlib

La compresión se realizará por medio de la libreria `Zlib`. Las funciones destacadas son: `compress` y `decompress`. La función `compress` requiere de los datos a comprimir, así como del nivel de compresión, el cual está comprendido entre 0 y 9 donde:

1\. El nivel 0 implica no compresión.

2\. El nivel 1 es el más rápido pero el que menos comprime. 

3\. El nivel 9 es el que más comprime pero es el más lento. 


## Buffers auxiliares

Dado que se van a realizar múltiples operaciones con vectores y con el propósito de obtener eficiencia, se sacrificará memoria, creando para ello dos matrices de numpy: una para el proceso de recepción y otra para el proceso de envío. El fundamiento de esta decisión, se basa en que los vectores de numpy son eficientes ya que tienen contiguas. 


## Slicing 

Se trata de un proceso de indexación proporcionado por numpy que permite extraer varios elementos de una matriz de numpy sin recurrir a bucles, obteniendo como consecuencia eficiencia. El proceso de indexación es de la forma: 

```Python
vector[start:end:step]
```


## Reorganización de las muestras


Se procede con la reestructuración de las muestras, de forma que sea agrupen todas las componentes de cada canal queden consecutivas. Esto genera taantos vectores como canales haya. Dichos vectores se concatenarán, y ese resultado será el que se comprima. De forma general este procedimiento se muestra en la siguiente figura. 



<center><img src="img/img1.svg" align="center" width = 500px length = 500px/></center>

&nbsp;

La siguiente figura, explica el proceso de compresión (`pack`) de forma más detallada. El proceso de compresión se realiza de la siguiente forma: 

1\. Se parte de una matriz de tantas columnas como canales y filas como frames. 

2\. Se separan los canales mediante slicing. Partiendo de dos canales, la forma de hacerlo es: 
``` Python
self.sender_chunk_buffer[0: len(self.sender_buf_size)//2] = chunk[:, 0]
self.sender_chunk_buffer[len(self.sender_buf_size)//2 : len(self.sender_chunk_buffer)] = chunk[:, 1]
``` 

3\. Se comprime el buffer de envio:

```Python
packed_chunk = zlib.compress(self.sender_chunk_buffer, self.compression_level)
````

4\. Se concatena el número de chunk con el resultado de la compresión:

```Python
packed_chunk = struct.pack("!H", chunk_number) + packed_chunk
```

5\. Se devuelve el chunk:

```Python
return packed_chunk 
```

<center><img src="img/compression.svg" align="center" width = 800px length = auto/></center>

&nbsp;

La siguiente figura representa el proceso de descompresión (`unpack`) de forma más detallada. El proceso de descompresión se realiza de la siguiente forma:  

1\. Se parte del paquete recibido por el socket.

2\. Se extrae el número de secuencia del chunk y el chunk comprimido: 

```Python
(chunk_number,) = struct.unpack("!H", packed_chunk[:2])
unpacked_chunk = packed_chunk[2:]
```

3\. Se descomprime el chunk y se pasa a vector de numpy:

```Python
unpacked_chunk = zlib.decompress(unpacked_chunk)
decompressed = np.frombuffer(unpacked_chunk, dtype=np.int16)
```

4\. Se reorganiza el vector generado para recuperar el mensaje original:

```Python
self.receiver_chunk_buffer[: , 0] = decompressed[0 : len(decompressed) // 2]
self.receiver_chunk_buffer[: , 1] = decompressed[(len(decompressed) // 2) : len(decompressed)]
```

5\. Se devuelve el número de chunk y el chunk descomprimido y organizado:

```Python
return chunk_number, self.receiver_chunk_buffer
```

<center><img src="img/decompression.svg" align="center" width = 800px length = auto/></center>

&nbsp;

# Consideraciones adicionales

Se presentan las siguientes consideraciones:


## Nivel de compresión

Añadimos un parámetro por línea de consola que nos permite especificar el nivel de compresión:

```Python
mini.parser.add_argument("-cl", "--compression_level", type=int, default=1, help="Compression level")
```
<b>NOTA:</b> Se debe comprobar que el valor introducido está dentro del rango permitido. 


## Modo no dual de canales

Se trabada con dos canales, pero podría no ser el caso. Si eso ocurre, el código generado no funcionará ya que está ajustado dos canales. Por este motivo, la manipulación de los buffers de envio y recepción se puede generalizar para un número arbitrario de canales. 

1. Generalización para la separación de canales en el método `pack`.

```Python
for i in range(0, mini.Minimal.NUMBER_OF_CHANNELS):
    self.sender_chunk_buffer[i  self.channel_size : (i + 1)  self.channel_size] = chunk[:, i]
```

2. Generalización para la separación de canales en el método `unpack`.

```Python
for i in range(0, mini.Minimal.NUMBER_OF_CHANNELS):
    self.receiver_chunk_buffer[: , i] = decompressed[i  self.channel_size  : (i+1)  self.channel_size]
```

# Documentación de compress.py

[`compress.py`](compress.html)